In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from utils import *
import anndata
import scanpy as sc
from pathlib import Path


/home/kristawang/miniconda3/envs/cytobulk/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def check_paths(output_folder,output_prefix=None):
    # Create relative path
    output_path = os.path.join(os.getcwd(), output_folder)

    # Make sure that the folder exists
    Path(output_path).mkdir(parents=True, exist_ok=True)

    if os.path.exists(os.path.join(output_path, f"{output_prefix}assigned_locations.csv")):
        print("\033[91mWARNING\033[0m: Running this will overwrite previous results, choose a new"
              " 'output_folder' or 'output_prefix'")

    return output_path

def read_file(file_path, file_label):
    '''
    Read data with given path.
    args:
        file_path: file path.
        file_label: the file label to raise exception.
    return:
        the read file.
    '''
    try:
        file_delim = "," if file_path.endswith(".csv") else "\t"
        with warnings.catch_warnings():
            file_data = pd.read_csv(file_path, sep=file_delim).dropna()
    except Exception as e:
        raise IOError (f"Make sure you provided the correct path to {file_label} files. "
                      "The following input file formats are supported: .csv with comma ',' as "
                      "delimiter, .txt or .tsv with tab '\\t' as delimiter.")
    return file_data

In [50]:
def read_single_cell_data(sc_path,meta_path,sc_nor,out_dir,save_ref=True):

    """read sc data and meta information.
    args:
        sc_path:    sc-rna data path.
        meta_path:  meta data with cell type information path.
        sc_nor:     Boolean, true for using preprocessing on sc data.
        out_path:   the dir to store the result files.
    """
    warnings.filterwarnings(action='ignore', category=FutureWarning) 
    warnings.filterwarnings(action='ignore', category=UserWarning)
    warnings.filterwarnings(action='ignore', category=RuntimeWarning)
    # read data
    print("Start to read training data and check the data format...")
    if sc_path.endswith(".h5ad"):
        sc_data = sc.read_h5ad(sc_path)
        sc_data = sc_data[sc_data.obs.Celltype_minor.isin(['B','Monocyte','NK','CD4Tn','M1','CD8Tex']),:]
        meta = pd.DataFrame(sc_data.obs,index=sc_data.obs_names)
        meta = meta.loc[sc_data.obs_names,:]
        meta.index.name = 'Cell'
        if isinstance(sc_data.X, np.ndarray):
            pass
        else:
            sc_data.X = sc_data.X.toarray()
        print("done")
    elif (sc_path is None) and (meta_path is None):
        raise ValueError("For sc data, you must provide either a .h5ad file or paths for expression and meta.")
    else:
        sc_data = read_file(sc_path,'training scRNA-seq')
        
        meta = read_file(meta_path,'training label')
        sc_data = anndata.AnnData(sc_data)
    if 'Celltype_major' or 'Celltype_minor' not in meta.columns:
        TypeError("The metadata need to have 'Celltype_major' and 'Celltype_minor' information")

    # format data
    sc_data = pd.DataFrame(sc_data.X.todense(),index=sc_data.obs_names,columns=sc_data.var_names).transpose()
    #sc_data.index.name = 'GeneSymbol'
    sc_data['GeneSymbol'] = sc_data.index
    sc_data['GeneSymbol'] = sc_data['GeneSymbol'].apply(lambda x: x.split('.')[0])
    sc_data=sc_data.groupby('GeneSymbol').mean()
    sc_data = sc_data.reset_index()
    sc_data = pd.DataFrame(sc_data.values[:,1:],index=sc_data['GeneSymbol'],columns=sc_data.columns[1:])
    cell_id = meta.index.tolist()
    common_id = list(set(cell_id) & set(sc_data.columns))
    sc_data = sc_data.loc[:,common_id]
    meta = meta.loc[common_id,:]
    print(f"Found {len(common_id)} valid cell")
    # save ref scrnaseq data
    out_dir = check_paths(out_dir+'/cell_feature')
    print('Saving ref scrna seq data...')
    save_ref = False
    if save_ref:
        for i in set(meta['Celltype_minor'].values.tolist()): 
            cell_id = meta[meta['Celltype_minor']==i].index.tolist()
            ref_sc = sc_data.loc[:,cell_id]
            ref_sc.to_csv(out_dir+f"/{i}_scrna.txt",sep='\t')

    return sc_data, meta

def get_stimulation(n_celltype,n_sample,meta,project,sc_data,out_dir,n,i_th=0,type='training'):
    """get stimulated expression data and cell type prop.
    args:
        type: string, training or testing.
    """
    print(f'The {type} data generation...')
    print("start to generate cell fraction...")
    cell_prop = np.random.dirichlet(np.ones(n_celltype), n_sample)
    # get cell meta dictionary
    meta_index = meta[['Celltype_minor']]
    meta_index = meta_index.groupby(meta['Celltype_minor']).groups
    for key, value in meta_index.items():
        meta_index[key] = np.array(value)
    cell_prop = cell_prop / np.sum(cell_prop, axis=1).reshape(-1, 1)
    print(f'The number of samples is {cell_prop.shape[0]}, the number of cell types is {cell_prop.shape[1]}, cell number is {n}')
    for i in range(int(cell_prop.shape[1])):
        indices = np.random.choice(np.arange(cell_prop.shape[0]), replace=False, size=int(cell_prop.shape[0] * 0.1))
        cell_prop[indices, i] = 0
    # get cell number based on cell prop
    print('Start sampling...')
    sample = np.zeros((cell_prop.shape[0],sc_data.shape[0]))
    allcellname = meta_index.keys()
    cell_prop = cell_prop / np.sum(cell_prop, axis=1).reshape(-1, 1)
    cell_num = np.floor(n * cell_prop)
    for i, sample_prop in tqdm(enumerate(cell_num)):
        for j, cellname in enumerate(allcellname):
            select_index = choice(meta_index[cellname], size=int(sample_prop[j]), replace=True)
            sample[i] += sc_data.loc[:,select_index].sum(axis=1)
    sample = sample/n
    print("Sampling down")
    sample = pd.DataFrame(sample,index=['Sample'+str(n_sample*(i_th)+i) for i in range(n_sample)],columns=sc_data.index.values)
    cell_prop = pd.DataFrame(cell_prop,index=['Sample'+str(n_sample*(i_th)+i) for i in range(n_sample)],columns=allcellname)
    return sample,cell_prop



def bulk_simulation(sc_path,meta_path,project,sc_nor,out_dir,n_sample=100,n=500,test=False):
    """get stimulated training data and testing data.
    args:
        test: boolean, true for generating testing data.
    """
    # read data
    start_t = time.perf_counter()
    sc_data, meta = read_single_cell_data(sc_path, meta_path, sc_nor, out_dir)
    print(f"Time to read and check training data: {round(time.perf_counter() - start_t, 2)} seconds")

    # training bulk rna
    train_out_dir = check_paths(out_dir+'/training_data')
    print('====================================================================')
    print('====================================================================')
    print('Start to stimulate the training bulk expression data ...')
    start_t = time.perf_counter()
    n_celltype = len(meta['Celltype_minor'].value_counts())
    new_data = []
    new_prop = []
    for i in range(int(n/100)):
        training_data,training_prop = get_stimulation(n_celltype,n_sample,meta,project,sc_data,train_out_dir,100*(i+1),i,type='training')
        new_data.append(training_data)
        new_prop.append(training_prop)
    training_data = pd.concat(new_data)
    training_prop = pd.concat(new_prop)
    training_prop.rename(columns={"B":"B cells","M1":"Macrophages M1","NK":"NK cells","CD4Tn":"T CD4 naive cells",
                                                  "CD8Tex":"T CD8 exhausted cells","Monocyte":"Monocytes"},inplace=True)
    
    print(f'Time to generate training data: {round(time.perf_counter() - start_t, 2)} seconds')

    return training_data,training_prop

In [51]:
training_sc_path = r"/mnt/d/project/CytoBulk/CytoBulk/data/NSCLC_GSE127471.h5ad"


In [52]:
training_data, training_prop= bulk_simulation(sc_path = training_sc_path,
                                                meta_path = "",
                                                project = "NSCLC_GSE127471",
                                                sc_nor = False,
                                                out_dir = "./test",
                                                n_sample = 50)

Start to read training data and check the data format...
done
Found 989 valid cell
Saving ref scrna seq data...
Time to read and check training data: 2.27 seconds
Start to stimulate the training bulk expression data ...
The training data generation...
start to generate cell fraction...
The number of samples is 50, the number of cell types is 6, cell number is 100
Start sampling...


50it [00:14,  3.38it/s]


Sampling down
The training data generation...
start to generate cell fraction...
The number of samples is 50, the number of cell types is 6, cell number is 200
Start sampling...


50it [00:17,  2.85it/s]


Sampling down
The training data generation...
start to generate cell fraction...
The number of samples is 50, the number of cell types is 6, cell number is 300
Start sampling...


...

Sampling down
The training data generation...
start to generate cell fraction...
The number of samples is 50, the number of cell types is 6, cell number is 400
Start sampling...


50it [00:22,  2.26it/s]


Sampling down
The training data generation...
start to generate cell fraction...
The number of samples is 50, the number of cell types is 6, cell number is 500
Start sampling...


50it [00:24,  2.02it/s]

Sampling down
Time to generate training data: 99.22 seconds


In [53]:
training_prop

,B cells,T CD4 naive cells,T CD8 exhausted cells,Macrophages M1,Monocytes,NK cells
Sample0,0.000000,0.081685,0.032337,0.683340,0.000000,0.202639
Sample1,0.000000,0.127641,0.000000,0.332838,0.306193,0.233328
Sample2,0.146582,0.083662,0.272163,0.197169,0.073879,0.226545
Sample3,0.102203,0.100671,0.373016,0.365184,0.040844,0.018083
Sample4,0.084241,0.000000,0.033593,0.529427,0.022402,0.330337
...,...,...,...,...,...,...
Sample245,0.149200,0.053832,0.018151,0.567827,0.209035,0.001955
Sample246,0.067729,0.202608,0.105839,0.052982,0.394417,0.176426
Sample247,0.099286,0.398998,0.104464,0.019025,0.088850,0.289378
Sample248,0.093964,0.121864,0.000000,0.493003,0.093337,0.197832


In [55]:
training_data.to_csv(r"/mnt/d/project/CytoBulk/CytoBulk/CytoBulk/test/training_data/NSCLC_GSE127471_expression.csv")
other_method_input = training_data.transpose()
other_method_input.index.name = 'GeneSymbol'
other_method_input.to_csv(r"/mnt/d/project/CytoBulk/CytoBulk/CytoBulk/test/training_data/other_training_data.csv")

In [54]:
training_prop.to_csv(r"/mnt/d/project/CytoBulk/CytoBulk/CytoBulk/test/training_data/NSCLC_GSE127471_fraction.csv")

In [27]:
training_data.to_csv(r"/mnt/d/project/CytoBulk/CytoBulk/CytoBulk/test/training_data/NSCLC_GSE127471_expression.csv")